In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4661051424629016649, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 199176327669285678
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10669279364384499725
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15882446439
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 18032932726806003114
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2, l1
from sklearn.metrics import cohen_kappa_score, accuracy_score

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.14.0


In [5]:
batch_size = 64
img_height, img_width = 450, 450
input_shape = (img_height, img_width, 3)
epochs = 1000

In [6]:
print(os.listdir("../input/aptos-multiclass-noaug/aptos_multiclass/"))

['test', 'train']


In [7]:
train_dir = '../input/aptos-multiclass-noaug/aptos_multiclass/train/'
test_dir = '../input/aptos-multiclass-noaug/aptos_multiclass/test/'

In [8]:
def preprocess_input(x):
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by imagenet mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

In [9]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
    preprocessing_function = preprocess_input,
    validation_split= 0.3)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255, preprocessing_function = preprocess_input)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 2325 images belonging to 5 classes.
Found 994 images belonging to 5 classes.
Found 343 images belonging to 5 classes.


In [10]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 2325
nb_validation_samples: 994
nb_test_samples: 343

predict_size_train: 37
predict_size_validation: 16
predict_size_test: 6

 num_classes: 5


In [11]:
os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "MobileNet_descriptors"

In [12]:
vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"
xception_weights = "../input/full-keras-pretrained-no-top/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [13]:
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception, preprocess_input
from keras.applications import DenseNet201
from keras.applications import DenseNet121
from keras.applications import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications import NASNetLarge, NASNetMobile
from keras.applications import MobileNet

In [14]:
input_tensor = Input(shape = input_shape)  

base_model1=DenseNet121(weights=denseNet121_weights, include_top=False, pooling = "avg")
# base_model2=Xception(input_shape= input_shape,weights=xception_weights, include_top=False, input_tensor=input_tensor)

# x1 = base_model1.output
# x1 = GlobalAveragePooling2D()(x1)

# x2 = base_model2.output
# x2 = GlobalAveragePooling2D()(x2)

# merge = concatenate([x1, x2])
# predictions = Dense(num_classes, activation='softmax')(merge)

# model = Model(inputs=input_tensor,outputs=predictions)

In [15]:
base_model1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [16]:
# for i, layer in enumerate(model.layers):
#     print(i, layer.name)

In [17]:
bottleneck_final_model = base_model1

In [18]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [19]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [20]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [21]:
# import tensorflow as tf
# from keras.backend.tensorflow_backend import get_session
# from keras.backend.tensorflow_backend import clear_session
# from keras.backend.tensorflow_backend import set_session

# def reset_keras_tf_session():
#     """
#     this function clears the gpu memory and set the 
#     tf session to not use the whole gpu
#     """
#     sess = get_session()
#     clear_session()
#     sess.close()
#     sess = get_session()

# #     config = tf.ConfigProto()
# #     config.gpu_options.allow_growth = True
# #     set_session(tf.Session(config=config))

# reset_keras_tf_session()

In [22]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [ ]:
adam_opt=Adam(lr=0.0001, beta_1=0.8, beta_2=0.9)

model = Sequential()
model.add(Dense(2048, activation="relu", kernel_regularizer=l2(1e-06), bias_regularizer=l2(0.01), activity_regularizer=l1(1e-07)))
model.add(Dropout(0.5))

model.add(Dense(1024, activation="relu", kernel_regularizer=l2(1e-06), bias_regularizer=l2(0.01), activity_regularizer=l1(1e-07)))
model.add(Dropout(0.5))

model.add(Dense(512, activation="relu", kernel_regularizer=l2(1e-06), bias_regularizer=l2(0.01), activity_regularizer=l1(1e-07)))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation="softmax"))

model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(validation_data, validation_labels),
                    verbose= 2)


Train on 2325 samples, validate on 994 samples
Epoch 1/1000
 - 3s - loss: 1.2166 - acc: 0.5583 - val_loss: 0.8628 - val_acc: 0.7093
Epoch 2/1000
 - 0s - loss: 0.9189 - acc: 0.6753 - val_loss: 0.7454 - val_acc: 0.7254
Epoch 3/1000
 - 0s - loss: 0.8152 - acc: 0.7127 - val_loss: 0.6872 - val_acc: 0.7586
Epoch 4/1000
 - 0s - loss: 0.7724 - acc: 0.7170 - val_loss: 0.6546 - val_acc: 0.7626
Epoch 5/1000
 - 0s - loss: 0.7308 - acc: 0.7325 - val_loss: 0.6411 - val_acc: 0.7666
Epoch 6/1000
 - 0s - loss: 0.6799 - acc: 0.7475 - val_loss: 0.6237 - val_acc: 0.7676
Epoch 7/1000
 - 0s - loss: 0.6610 - acc: 0.7544 - val_loss: 0.6128 - val_acc: 0.7807
Epoch 8/1000
 - 0s - loss: 0.6303 - acc: 0.7639 - val_loss: 0.6015 - val_acc: 0.7817
Epoch 9/1000
 - 0s - loss: 0.6276 - acc: 0.7695 - val_loss: 0.5900 - val_acc: 0.7867
Epoch 10/1000
 - 0s - loss: 0.6247 - acc: 0.7682 - val_loss: 0.5849 - val_acc: 0.7767
Epoch 11/1000
 - 0s - loss: 0.6074 - acc: 0.7647 - val_loss: 0.5763 - val_acc: 0.7807
Epoch 12/1000
 -

Epoch 96/1000
 - 0s - loss: 0.1532 - acc: 0.9462 - val_loss: 0.7227 - val_acc: 0.8310
Epoch 97/1000
 - 0s - loss: 0.1435 - acc: 0.9505 - val_loss: 0.7737 - val_acc: 0.8209
Epoch 98/1000
 - 0s - loss: 0.1612 - acc: 0.9381 - val_loss: 0.7612 - val_acc: 0.8068
Epoch 99/1000
 - 0s - loss: 0.1560 - acc: 0.9445 - val_loss: 0.7676 - val_acc: 0.8189
Epoch 100/1000
 - 0s - loss: 0.1452 - acc: 0.9527 - val_loss: 0.7426 - val_acc: 0.8058
Epoch 101/1000
 - 0s - loss: 0.1631 - acc: 0.9445 - val_loss: 0.7469 - val_acc: 0.8149
Epoch 102/1000
 - 0s - loss: 0.1470 - acc: 0.9467 - val_loss: 0.7739 - val_acc: 0.8159
Epoch 103/1000
 - 0s - loss: 0.1461 - acc: 0.9471 - val_loss: 0.8268 - val_acc: 0.7918
Epoch 104/1000
 - 0s - loss: 0.1418 - acc: 0.9471 - val_loss: 0.8062 - val_acc: 0.8149
Epoch 105/1000
 - 0s - loss: 0.1437 - acc: 0.9505 - val_loss: 0.7484 - val_acc: 0.8249
Epoch 106/1000
 - 0s - loss: 0.1302 - acc: 0.9600 - val_loss: 0.8079 - val_acc: 0.8109
Epoch 107/1000
 - 0s - loss: 0.1228 - acc: 0.95

Epoch 191/1000
 - 0s - loss: 0.0660 - acc: 0.9772 - val_loss: 1.1965 - val_acc: 0.7968
Epoch 192/1000
 - 0s - loss: 0.0679 - acc: 0.9802 - val_loss: 1.1025 - val_acc: 0.8149
Epoch 193/1000
 - 0s - loss: 0.0866 - acc: 0.9763 - val_loss: 1.1459 - val_acc: 0.8219
Epoch 194/1000
 - 0s - loss: 0.0654 - acc: 0.9806 - val_loss: 1.2344 - val_acc: 0.8048
Epoch 195/1000
 - 0s - loss: 0.0927 - acc: 0.9682 - val_loss: 1.0497 - val_acc: 0.8229
Epoch 196/1000
 - 0s - loss: 0.0745 - acc: 0.9751 - val_loss: 1.0771 - val_acc: 0.8139
Epoch 197/1000
 - 0s - loss: 0.0729 - acc: 0.9746 - val_loss: 1.1164 - val_acc: 0.8149
Epoch 198/1000
 - 0s - loss: 0.0786 - acc: 0.9738 - val_loss: 1.0689 - val_acc: 0.8189
Epoch 199/1000
 - 0s - loss: 0.0761 - acc: 0.9768 - val_loss: 1.1096 - val_acc: 0.8048
Epoch 200/1000
 - 0s - loss: 0.0748 - acc: 0.9751 - val_loss: 1.0913 - val_acc: 0.8099
Epoch 201/1000
 - 0s - loss: 0.0781 - acc: 0.9716 - val_loss: 1.0899 - val_acc: 0.8179
Epoch 202/1000
 - 0s - loss: 0.0630 - acc: 

 - 0s - loss: 0.0543 - acc: 0.9819 - val_loss: 1.2560 - val_acc: 0.8078
Epoch 286/1000
 - 0s - loss: 0.0819 - acc: 0.9755 - val_loss: 1.1980 - val_acc: 0.8159
Epoch 287/1000
 - 0s - loss: 0.0570 - acc: 0.9819 - val_loss: 1.1865 - val_acc: 0.8099
Epoch 288/1000
 - 0s - loss: 0.0787 - acc: 0.9755 - val_loss: 1.1437 - val_acc: 0.8249
Epoch 289/1000
 - 0s - loss: 0.0577 - acc: 0.9789 - val_loss: 1.3483 - val_acc: 0.8109
Epoch 290/1000
 - 0s - loss: 0.0697 - acc: 0.9776 - val_loss: 1.2104 - val_acc: 0.8089
Epoch 291/1000
 - 0s - loss: 0.0664 - acc: 0.9772 - val_loss: 1.1159 - val_acc: 0.8159
Epoch 292/1000
 - 0s - loss: 0.0601 - acc: 0.9806 - val_loss: 1.2102 - val_acc: 0.7998
Epoch 293/1000
 - 0s - loss: 0.0609 - acc: 0.9832 - val_loss: 1.1913 - val_acc: 0.8159
Epoch 294/1000
 - 0s - loss: 0.0590 - acc: 0.9811 - val_loss: 1.2135 - val_acc: 0.8169
Epoch 295/1000
 - 0s - loss: 0.0546 - acc: 0.9802 - val_loss: 1.3611 - val_acc: 0.8109
Epoch 296/1000
 - 0s - loss: 0.0676 - acc: 0.9768 - val_lo

Epoch 380/1000
 - 0s - loss: 0.0632 - acc: 0.9789 - val_loss: 1.3054 - val_acc: 0.8229
Epoch 381/1000
 - 0s - loss: 0.0647 - acc: 0.9763 - val_loss: 1.2758 - val_acc: 0.8139
Epoch 382/1000
 - 0s - loss: 0.0524 - acc: 0.9828 - val_loss: 1.2647 - val_acc: 0.8119
Epoch 383/1000
 - 0s - loss: 0.0529 - acc: 0.9794 - val_loss: 1.3423 - val_acc: 0.8149
Epoch 384/1000
 - 0s - loss: 0.0685 - acc: 0.9781 - val_loss: 1.1711 - val_acc: 0.8099
Epoch 385/1000
 - 0s - loss: 0.0519 - acc: 0.9811 - val_loss: 1.2825 - val_acc: 0.8159
Epoch 386/1000
 - 0s - loss: 0.0499 - acc: 0.9828 - val_loss: 1.3648 - val_acc: 0.8119
Epoch 387/1000
 - 0s - loss: 0.0507 - acc: 0.9832 - val_loss: 1.3869 - val_acc: 0.8119
Epoch 388/1000
 - 0s - loss: 0.0718 - acc: 0.9789 - val_loss: 1.2998 - val_acc: 0.8149
Epoch 389/1000
 - 0s - loss: 0.0539 - acc: 0.9832 - val_loss: 1.2294 - val_acc: 0.8099
Epoch 390/1000
 - 0s - loss: 0.0483 - acc: 0.9819 - val_loss: 1.3576 - val_acc: 0.8179
Epoch 391/1000
 - 0s - loss: 0.0707 - acc: 

 - 0s - loss: 0.0623 - acc: 0.9776 - val_loss: 1.3123 - val_acc: 0.8159
Epoch 475/1000
 - 0s - loss: 0.0530 - acc: 0.9811 - val_loss: 1.2795 - val_acc: 0.8179
Epoch 476/1000
 - 0s - loss: 0.0376 - acc: 0.9867 - val_loss: 1.5190 - val_acc: 0.8089
Epoch 477/1000
 - 0s - loss: 0.0548 - acc: 0.9794 - val_loss: 1.3975 - val_acc: 0.8129
Epoch 478/1000
 - 0s - loss: 0.0494 - acc: 0.9824 - val_loss: 1.2399 - val_acc: 0.8139
Epoch 479/1000
 - 0s - loss: 0.0485 - acc: 0.9854 - val_loss: 1.3720 - val_acc: 0.8149
Epoch 480/1000
 - 0s - loss: 0.0561 - acc: 0.9811 - val_loss: 1.3704 - val_acc: 0.8099
Epoch 481/1000
 - 0s - loss: 0.0468 - acc: 0.9828 - val_loss: 1.2754 - val_acc: 0.8229
Epoch 482/1000
 - 0s - loss: 0.0461 - acc: 0.9867 - val_loss: 1.4567 - val_acc: 0.8139
Epoch 483/1000
 - 0s - loss: 0.0500 - acc: 0.9837 - val_loss: 1.3396 - val_acc: 0.8189
Epoch 484/1000
 - 0s - loss: 0.0643 - acc: 0.9794 - val_loss: 1.3222 - val_acc: 0.8179
Epoch 485/1000
 - 0s - loss: 0.0480 - acc: 0.9854 - val_lo

In [ ]:
(eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
print("Validation Loss: {}".format(eval_loss))

In [ ]:
filename = test_generator.filenames
truth = test_generator.classes
label = test_generator.class_indices
indexlabel = dict((value, key) for key, value in label.items())

from sklearn.metrics import accuracy_score, confusion_matrix
preds = model.predict(test_data)

predictions = [i.argmax() for i in preds]
y_true = [i.argmax() for i in test_labels]
cm = confusion_matrix(y_pred=predictions, y_true=y_true)

print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

In [ ]:
plt.rcParams["axes.grid"] = False
plt.rcParams.update({'font.size': 20})

labels = []

label = test_generator.class_indices
indexlabel = dict((value, key) for key, value in label.items())

for k,v in indexlabel.items():
    labels.append(v)

from sklearn.metrics import confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion Matrix')

    print(cm)
#     fig = plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
#     plt.savefig('plots/3.InceptionV3-2-Private-DataSet-CM.png', bbox_inches='tight', dpi = 100) 


plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, classes=labels, title=' ')

In [ ]:
cohen_kappa_score(y_true, y_pred)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
y_pred=predictions
y_pred_probabilities=y_pred

# y_pred = np.argmax(y_pred,axis = 1) 
y_actual = y_true

classnames=[]
for classname in test_generator.class_indices:
    classnames.append(classname)

confusion_mtx = confusion_matrix(y_actual, y_pred) 
print(confusion_mtx)
target_names = classnames
print(classification_report(y_actual, y_pred, target_names=target_names))

In [ ]:
total=sum(sum(cm))

sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])
print('Sensitivity : ', sensitivity )

Specificity = cm[1,1]/(cm[1,1]+cm[0,1])
print('Specificity : ', Specificity )

In [ ]:
y_pred_class = model.predict(test_data, verbose=1)

y_pred_class = [np.argmax(r) for r in y_pred_class]
test_y = [np.argmax(r) for r in test_labels]


from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, roc_curve

# Precision
print('Precision = ', precision_score(test_y, y_pred_class, average='weighted'))
# (None, 'micro', 'macro', 'weighted', 'samples')

# Recall
print('Recall = ', recall_score(test_y, y_pred_class, average='weighted'))

# f1_score
print('f1_score = ', f1_score(test_y, y_pred_class, average='weighted'))

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def multiclass_roc_auc_score(y_test, y_pred, average="weighted"):
    label_binarizer = LabelBinarizer()
    label_binarizer.fit(y_test)

    truth = label_binarizer.transform(y_test)
    pred = label_binarizer.transform(y_pred)
    return roc_auc_score(truth, pred, average=average)
# roc_auc_score
print('roc_auc_score = ', multiclass_roc_auc_score(test_y, y_pred_class))

In [ ]:
plt.rcParams.update({'font.size': 12})

import seaborn
plt.style.use('seaborn-white')

plt.figure()
N = epochs
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")

plt.legend(loc="center right")